In [1]:
%matplotlib inline
import math
import time
import numpy as np
import random
import torch
from d2l import torch as d2l

In [2]:
n = 10000
a = torch.ones(n)
b = torch.ones(n)

In [3]:
class Timer:
    """Record running time."""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        self.tik = time.time()
    
    def stop(self):
        self.times.append(time.time() - self.tik)
        return self.times[-1]
    
    def avg(self):
        return sum(self.times) / len(self.times)

    def sum(self):
        return sum(self.times)

    def cumsum(self):
        return np.array(self.times).cumsum.tolist()

In [4]:
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]
print(f'{timer.stop():.5f} sec')
timer.start()
d = a + b
print(f'{timer.stop():.5f} sec')

0.10000 sec
0.00000 sec


In [5]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1,1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2

features, labels = synthetic_data(true_w, true_b, 1000)

In [6]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

Define model, loss, optimizer

In [7]:
def linreg(X, w, b):
    return torch.matmul(X, w) + b
    
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [8]:
w = torch.normal(0,0.01, size=(2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
batch_size = 10

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.034071
epoch 2, loss 0.000120
epoch 3, loss 0.000049


In [9]:
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: tensor([ 0.0003, -0.0001], grad_fn=<SubBackward0>)
error in estimating b: tensor([0.0009], grad_fn=<RsubBackward1>)


Use Predefined Pipeline

In [10]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [18]:
def load_array(data_arrays, batch_size, is_train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = is_train)

batch_size = 10

data_iter = load_array((features, labels), batch_size)


In [44]:
next(iter(data_iter))

[tensor([[ 1.8824, -1.2965],
         [-0.2764, -1.0385],
         [ 0.5898,  0.6036],
         [-0.2910,  1.9095],
         [-2.0905,  1.1339],
         [-0.6604, -0.8904],
         [ 0.2135,  1.3084],
         [-0.2056,  0.3431],
         [ 0.1896,  1.1763],
         [ 0.7993,  2.2512]]),
 tensor([[12.3798],
         [ 7.1812],
         [ 3.3324],
         [-2.8547],
         [-3.8138],
         [ 5.9252],
         [ 0.1809],
         [ 2.6213],
         [ 0.5876],
         [-1.8421]])]

In [49]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))


In [50]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [51]:
loss = nn.MSELoss()

In [53]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [54]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000225
epoch 2, loss 0.000103
epoch 3, loss 0.000102
